In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X, Y = make_classification(n_samples=1000, n_features=20, n_classes=5, n_clusters_per_class=1,n_informative=15, n_redundant=0, random_state=42)
encoded_label=OneHotEncoder(sparse=False).fit_transform(Y.reshape(-1,1))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
np.set_printoptions(threshold=np.inf)
print(X_train)
print(Y_train)

[[ 6.72329863e-02  3.11754922e+00 -1.25670685e+00  3.13842808e-01
   1.96713797e+00 -6.71735760e-02  7.39196768e-01  3.00119898e+00
  -1.41229956e+00 -4.96849261e-01  1.38656570e+00  7.40398603e-01
   4.77169909e-01 -1.23471456e+00  3.93180684e+00 -4.92236501e-01
  -2.01494233e+00 -7.49174748e-01 -9.85456788e-01 -6.96153038e-01]
 [ 6.51046375e-01 -1.75867825e+00  4.96804899e-01  2.56019723e+00
   5.45381066e-01 -1.91569226e-01  1.18280008e+00 -1.43711735e+00
  -3.59084067e+00  1.95154672e+00 -2.29692939e+00  7.71505245e-01
   1.89727252e-01 -3.01099770e-01 -2.37351474e-01  2.63570045e-02
  -3.20617706e+00 -3.10101954e-01 -9.44687656e-01 -1.93074665e+00]
 [-2.01015709e-01 -7.87219763e-01  3.25267641e-01  4.64041701e+00
   2.35475985e+00  1.61123732e+00  1.42650414e+00 -8.58529791e-02
  -2.57495365e+00  4.38221457e+00 -2.55242645e+00  5.48682565e+00
   1.41175814e+00  1.95260001e+00  1.07224265e+00  4.05752278e-01
  -4.83815659e+00  7.76795282e-01 -2.85463347e-01 -1.63113434e-01]
 [-2.33

In [4]:
class NeuralNetwork(object):
    def __init__(self):
        inputLayerNeurons = 20  # Input features
        hiddenLayerNeurons1 = 64
        hiddenLayerNeurons2 = 32
        hiddenLayerNeurons3 = 16
        outLayerNeurons = 5    # Number of classes

        self.learning_rate = 0.01
        self.W_H1 = np.random.randn(inputLayerNeurons, hiddenLayerNeurons1)
        self.W_H2 = np.random.randn(hiddenLayerNeurons1, hiddenLayerNeurons2)
        self.W_H3 = np.random.randn(hiddenLayerNeurons2, hiddenLayerNeurons3)
        self.W_O = np.random.randn(hiddenLayerNeurons3, outLayerNeurons)

    def sigmoid(self, x, der=False):
        if der:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_true, y_pred):
        m = y_true.shape[0]
        p = y_pred
        log_likelihood = -np.log(p[range(m), np.argmax(y_true, axis=1)])
        loss = np.sum(log_likelihood) / m
        return loss

    def feedForward(self, X):
        self.hidden_output1 = self.sigmoid(np.dot(X, self.W_H1))
        self.hidden_output2 = self.sigmoid(np.dot(self.hidden_output1, self.W_H2))
        self.hidden_output3 = self.sigmoid(np.dot(self.hidden_output2, self.W_H3))
        output_input = np.dot(self.hidden_output3, self.W_O)
        self.output = self.softmax(output_input)
        return self.output

    def backPropagation(self, X, Y, pred):
        m = Y.shape[0]

        # Calculate gradients
        output_error = pred - Y
        output_delta = output_error / m

        hidden_error3 = output_delta.dot(self.W_O.T)
        hidden_delta3 = hidden_error3 * self.sigmoid(self.hidden_output3, der=True)

        hidden_error2 = hidden_delta3.dot(self.W_H3.T)
        hidden_delta2 = hidden_error2 * self.sigmoid(self.hidden_output2, der=True)

        hidden_error1 = hidden_delta2.dot(self.W_H2.T)
        hidden_delta1 = hidden_error1 * self.sigmoid(self.hidden_output1, der=True)

        # Update weights
        self.W_O -= self.learning_rate * self.hidden_output3.T.dot(output_delta)
        self.W_H3 -= self.learning_rate * self.hidden_output2.T.dot(hidden_delta3)
        self.W_H2 -= self.learning_rate * self.hidden_output1.T.dot(hidden_delta2)
        self.W_H1 -= self.learning_rate * X.T.dot(hidden_delta1)

    def train(self, X, Y, epochs=10000):
        for epoch in range(epochs):
            pred = self.feedForward(X)
            self.backPropagation(X, Y, pred)
            loss = self.cross_entropy_loss(Y, pred)
            if epoch % 1000 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')

    def predict(self, X):
        pred = self.feedForward(X)
        return np.argmax(pred, axis=1)


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, encoded_label, test_size=0.2, random_state=42)
# Train the Neural Network
NN = NeuralNetwork()
NN.train(X_train, Y_train)

# Predictions
train_predictions = NN.predict(X_train)
test_predictions = NN.predict(X_test)

# Calculate performance metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

train_accuracy = accuracy_score(np.argmax(Y_train, axis=1), train_predictions)
test_accuracy = accuracy_score(np.argmax(Y_test, axis=1), test_predictions)

print(f'Training Accuracy: {train_accuracy}')
print(f'Test Accuracy: {test_accuracy}')

print("Classification Report on Test Set:")
print(classification_report(np.argmax(Y_test, axis=1), test_predictions))

print("Confusion Matrix:")
print(confusion_matrix(np.argmax(Y_test, axis=1), test_predictions))


Epoch 0, Loss: 2.9370656405812143
Epoch 1000, Loss: 1.2847102964394512
Epoch 2000, Loss: 1.007740852471839
Epoch 3000, Loss: 0.8327558402571126
Epoch 4000, Loss: 0.7114952552514072
Epoch 5000, Loss: 0.6220248074911724
Epoch 6000, Loss: 0.5516609179397968
Epoch 7000, Loss: 0.4936104084796213
Epoch 8000, Loss: 0.4443387446186941
Epoch 9000, Loss: 0.402151620917906
Training Accuracy: 0.89875
Test Accuracy: 0.76
Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.78      0.64      0.70        44
           1       0.74      0.85      0.79        34
           2       0.87      0.79      0.83        52
           3       0.71      0.86      0.78        37
           4       0.67      0.67      0.67        33

    accuracy                           0.76       200
   macro avg       0.75      0.76      0.75       200
weighted avg       0.77      0.76      0.76       200

Confusion Matrix:
[[28  3  1  5  7]
 [ 2 29  0  0  3]
 [ 3  2 41

In [8]:
input_data = np.array([-3.3946745, -2.84888078,  2.808603, -7.19786, -4.2754,  7.96877, -1.2712, -1.48187, -4.10588,  3.918323, -1.25374,  1.2748662, -6.84099, -3.150453, -1.729752,  1.151189, -5.981176,  4.047248, -1.306079,  1.036481])
input_data = input_data.reshape(1, -1)  # Reshape to (1, number_of_features)

# Feed the reshaped input to the network
output = NN.feedForward(input_data)
print(output)

[[4.05139161e-04 5.58980997e-04 9.73168156e-01 2.51596179e-02
  7.08105616e-04]]
